In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize

# Load saved models and encoders
clf = joblib.load('/content/drive/MyDrive/modelsaved/fitness_goal_classifier.pkl')
regressor = joblib.load('/content/drive/MyDrive/modelsaved/calories_burned_regressor.pkl')
scaler = joblib.load('/content/drive/MyDrive/modelsaved/scaler.pkl')
label_encoders = joblib.load('/content/drive/MyDrive/modelsaved/label_encoders.pkl')

# Features for classification and regression
level1_features = ['Age', 'Gender', 'BMI', 'Workout_Frequency (days/week)',
                   'Experience_Level', 'Workout_Type', 'Fat_Percentage']
level2_features = ['Age', 'Gender', 'Weight (kg)', 'Height (m)', 'Max_BPM', 'Avg_BPM', 'Resting_BPM',
                   'Session_Duration (hours)', 'Workout_Type', 'Fat_Percentage', 'BMI', 'Predicted_Fitness_Goal']

# Define optimization function for Level 3 with dynamic adjustments
def objective(workout_plan, alpha=0.6, beta=0.3, gamma=0.1, user_experience=1, predicted_calories=1, fitness_goal=None):
    session_duration, target_bpm, fatigue_risk = workout_plan
    adjusted_alpha = alpha * user_experience
    adjusted_beta = beta * (target_bpm / 200)
    adjusted_gamma = gamma * (1 + fatigue_risk)

    # Adjust goal alignment based on predicted fitness goal
    goal_alignment = 0
    if fitness_goal == 'Weight_Loss':
        goal_alignment = 1  # prioritize higher calories burnt for weight loss
    elif fitness_goal == 'Muscle_Gain':
        goal_alignment = 0.8  # slightly less focus on calories burnt
    else:
        goal_alignment = 0.9  # balance for endurance

    # Add penalty for long session durations if user has low experience
    penalty_duration = 0 if session_duration <= 1 + user_experience * 0.2 else session_duration * 0.5

    return -(adjusted_alpha * predicted_calories * goal_alignment - adjusted_beta - adjusted_gamma + penalty_duration)

def constraint_duration(workout_plan, user_experience):
    session_duration = workout_plan[0]
    max_duration = 0.5 + user_experience * 0.3
    return max_duration - session_duration

def constraint_bpm(workout_plan, max_bpm, age):
    target_bpm = workout_plan[1]
    safe_bpm = max_bpm - (0.5 * (age - 30))
    return safe_bpm - target_bpm

def get_initial_plan(user_experience):
    return [np.random.uniform(0.5, 2.0), np.random.uniform(100, 180), np.random.uniform(0.0, 1.0)]

bounds = [(0.5, 2.0), (100, 200), (0.0, 1.0)]

# Function to calculate the optimal plan based on user input
def calculate_optimal_plan(user_experience=1, max_bpm=200, predicted_calories=1, age=30, fitness_goal=None):
    initial_plan = get_initial_plan(user_experience)
    result = minimize(objective, initial_plan, bounds=bounds,
                      constraints=[
                          {"type": "ineq", "fun": lambda workout_plan: constraint_duration(workout_plan, user_experience)},
                          {"type": "ineq", "fun": lambda workout_plan: constraint_bpm(workout_plan, max_bpm, age)}
                      ], args=(0.6, 0.3, 0.1, user_experience, predicted_calories, fitness_goal))
    return result.x

# Function to calculate water intake
def calculate_water_intake(weight, session_duration):
    base_water = 0.03 * weight
    extra_water = session_duration * 0.5
    return base_water + extra_water

# Function to make predictions based on user input
def interactive_prediction():
    print("Please enter the following details:")

    # Taking user input
    age = int(input("Age: "))
    gender = input("Gender (Male/Female): ").strip()
    bmi = float(input("BMI: "))
    workout_frequency = int(input("Workout Frequency (days/week): "))
    experience_level = int(input("Experience Level (1-5): "))
    workout_type = input("Workout Type (Cardio/Strength/Other): ").strip()
    fat_percentage = float(input("Fat Percentage: "))
    weight = float(input("Weight (kg): "))
    height = float(input("Height (m): "))
    max_bpm = int(input("Max BPM: "))
    avg_bpm = int(input("Average BPM: "))
    resting_bpm = int(input("Resting BPM: "))
    session_duration = float(input("Session Duration (hours): "))

    # Encoding gender and workout type
    gender_encoded = label_encoders['Gender'].transform([gender])[0]
    workout_type_encoded = label_encoders['Workout_Type'].transform([workout_type])[0]

    # Level 1 Prediction (Fitness Goal)
    level1_input = pd.DataFrame([[age, gender_encoded, bmi, workout_frequency,
                                  experience_level, workout_type_encoded, fat_percentage]],
                                columns=level1_features)
    fitness_goal_pred = clf.predict(level1_input)[0]
    fitness_goal_label = label_encoders['Fitness_Goal'].inverse_transform([fitness_goal_pred])[0]

    # Level 2 Prediction (Calories Burned)
    level2_input = scaler.transform([[age, gender_encoded, weight, height, max_bpm, avg_bpm,
                                       resting_bpm, session_duration, workout_type_encoded,
                                       fat_percentage, bmi, fitness_goal_pred ]])
    calories_burned_pred = regressor.predict(level2_input)[0]

    # Level 3 Optimization (Workout Plan)
    optimal_plan = calculate_optimal_plan(user_experience=experience_level, max_bpm=max_bpm, predicted_calories=calories_burned_pred, age=age, fitness_goal=fitness_goal_label)
    optimal_bpm = optimal_plan[1]
    if optimal_bpm > 160:
        optimal_workout_type = 'HIIT'
    elif optimal_bpm > 140:
        optimal_workout_type = 'Cardio'
    elif optimal_bpm > 120:
        optimal_workout_type = 'Strength'
    else:
        optimal_workout_type = 'Endurance'

    recommended_water = calculate_water_intake(weight, optimal_plan[0])

    # Output results
    print("\nPredicted Fitness Goal (Level 1):", fitness_goal_label)
    print("Predicted Calories Burned (Level 2): {:.2f} kcal".format(calories_burned_pred))
    print("\nOptimal Workout Plan (Level 3):")
    print("Session Duration: {:.2f} hours".format(optimal_plan[0]))
    print("Target BPM: {:.1f}".format(optimal_plan[1]))
    print("Fatigue Risk: {:.2f}".format(optimal_plan[2]))
    print("Recommended Workout Type:", optimal_workout_type)
    print("Recommended Water Intake: {:.2f} liters".format(recommended_water))

# Run interactive prediction
interactive_prediction()


Please enter the following details:
Age: 70
Gender (Male/Female): Male
BMI: 33.4
Workout Frequency (days/week): 1
Experience Level (1-5): 1
Workout Type (Cardio/Strength/Other): Strength
Fat Percentage: 20.4
Weight (kg): 66
Height (m): 1.65
Max BPM: 130
Average BPM: 90
Resting BPM: 40
Session Duration (hours): 1.66

Predicted Fitness Goal (Level 1): Muscle_Gain
Predicted Calories Burned (Level 2): 1023.66 kcal

Optimal Workout Plan (Level 3):
Session Duration: 0.54 hours
Target BPM: 100.0
Fatigue Risk: 0.00
Recommended Workout Type: Endurance
Recommended Water Intake: 2.25 liters


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
